## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [5]:
%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
# %serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-02U1W54L --baud=115200 
MicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [4]:
%sendtofile lib/runner.py --source lib/runner.py

Sent 246 lines (8210 bytes) to lib/runner.py.


In [15]:
%sendtofile lib/synthetic.py --source lib/synthetic.py

Sent 47 lines (1573 bytes) to lib/synthetic.py.


In [3]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 238 lines (8870 bytes) to lib/decoding.py.


### Using a Runner for experimentation and logging
The a `Runner` is encapsulates the core functions in this EEG system, including peripheral setup, sampling, signal processing, logging and memory management. The `OnlineRunner` offers mostly the same functionality as the standard `Runner` class, except it allows for logging and other communication with a remote server - either on the Internet or on your local network.

#### Offline functionality
The standard `Runner` is good for testing core functionality without the need for remote logging. See below for initialisation and execution.

In [23]:
from lib.runner import OnlineRunner, Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

decoding_algo = 'MsetCCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62497708393011


In [24]:
from lib.synthetic import synth_X

calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
runner.calibrate(calibration_data)

In [25]:
# start sampling and recording data (logging not setup in this case)
runner.run()

In [26]:
# see if runner has indeed started smapling
print(runner.is_sampling)

True


In [27]:
# display the contents of the output buffer - this will be updated internally by the runner
# at a rate determined by the sampling frequency and sample buffer size (typically every 1s)

print(runner.output_buffer)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [33]:
# decode the contents of the output buffer. This will return a dictionary whose keys are the
# candidate SSVEP stimulus frequencies and the values, their corresponding output correlation
# estimate

print(runner.decode())

{12: 0.09387, 10: 0.11474, 7: 0.05861999999999999}


In [30]:
# stop runner
runner.stop()

#### Simple decoding loop
In order to test online decoding, here is a basic synchronous loop-based option. Interrupt the cell to stop the infinite loop.

In [73]:
import utime as time
from lib.runner import Runner

decode_period_s = 2 # decode every x seconds

runner = Runner() # initialise a base runner
runner.setup()

runner.run() # start sampling

try:
    while True:
        time.sleep(decode_period_s)
        print(runner.decode())
except KeyboardInterrupt:
    print('received SIGINT - stopping')

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
{12: 0.12781, 10: 0.13684, 7: 0.12109}
.{12: 0.10859, 10: 0.17859, 7: 0.13157}
{12: 0.16485, 10: 0.08684, 7: 0.08995}
{12: 0.11967, 10: 0.12927, 7: 0.15572}
.{12: 0.17864, 10: 0.16814, 7: 0.15376}
{12: 0.18824, 10: 0.12346, 7: 0.17182}
.{12: 0.12428, 10: 0.13551, 7: 0.1034}
{12: 0.08138, 10: 0.05915, 7: 0.11145}
.{12: 0.12305, 10: 0.04883, 7: 0.14783}


*** Sending Ctrl-C

received SIGINT - stopping


### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [ ]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

#### Online Runner
Now that you've established network connectivitiy, you can test out an `OnlineRunner`. In order to test web logging to a remote server, we can use a basic HTTP logger. However, this obviously needs an API/server willing to accept our requests. There is a basic logging API using `Flask` in `/eeg_lib/logging_server.py`. You can run it using `python logging_server.py` which will spin up a development server on the predefined port (5000 or 5001). Then, just configure your `OnlineRunner` with the appropriate logger params and you're set.

In [19]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://192.168.0.2:5001/" # make sure the port corresponds to your logging server configuration
log_params = dict(server=api_host, log_period=4, logger_type=logger_types.HTTP, send_raw=True, session_id='test_session_1')

runner = OnlineRunner()
runner.setup(**log_params)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.28', '255.255.255.0', '192.168.0.1', '192.168.0.1')


In [20]:
# start the runner - you should see requests being made to your local server
runner.run()

In [21]:
runner.stop()

## Experimentation

In [155]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [12]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US